In [2]:
import numpy as np
import pandas as pd
import math
import time
import pyvisa
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

# Medir

In [ ]:

def guardar_mediciones():
    # Mediciones_CH1.csv
    df_1 = pd.DataFrame([freq_gen_fun, Unidades_señales_CH1])
    df_1nuevo = pd.DataFrame(señales_CH1).T
    df_mediciones_CH1 = pd.concat([df_1, df_1nuevo], ignore_index=True)
    titulos_filas = ["frecuencias generador funciones (Hz)", "Unidades señales CH1", "Señales CH1"]
    while len(titulos_filas) < len(df_mediciones_CH1):
        titulos_filas.append("")
    df_mediciones_CH1.insert(0, "Títulos de Fila", titulos_filas)
    df_mediciones_CH1.to_csv(r'C:.\Mediciones_CH1.csv', index=False, encoding = 'utf-8')
    time.sleep(0.5)

    # Mediciones_CH2.csv
    df_2 = pd.DataFrame([freq_gen_fun, Unidades_señales_CH2])
    df_2nuevo = pd.DataFrame(señales_CH2).T
    df_mediciones_CH2 = pd.concat([df_2, df_2nuevo], ignore_index=True)
    titulos_filas_2 = ["frecuencias generador funciones (Hz)", "Unidades señales CH2", "Señales CH2"]
    while len(titulos_filas_2) < len(df_mediciones_CH2):
        titulos_filas_2.append("")
    df_mediciones_CH2.insert(0, "Títulos de Fila", titulos_filas_2)
    df_mediciones_CH2.to_csv(r'C:.\Mediciones_CH2.csv', index=False, encoding = 'utf-8')
    time.sleep(0.5)

    #Unidades y escala.csv
    df_unidades_y_escala = pd.DataFrame([freq_gen_fun, Unidades_señales_CH1, Unidades_tiempos_CH1, Escala_señales_CH1, Escala_tiempos_CH1, Unidades_señales_CH2, Unidades_tiempos_CH2, Escala_señales_CH2, Escala_tiempos_CH2 ])
    titulos_filas_3 = ["frecuencias generador funciones (Hz)", "Unidades señales Ch1", "Unidades tiempos Ch1", "Escala señales Ch1 (V)", "Escala Tiempos Ch1 (s)", "Unidades señales Ch2", "Unidades tiempos Ch2", "Escala señales Ch2 (V)", "Escala Tiempos Ch2 (s)"]
    df_unidades_y_escala.insert(0, "Títulos de Fila", titulos_filas_3)
    df_unidades_y_escala.to_csv(r'C:.\Unidades y escala.csv', index=False, encoding = 'utf-8')
    time.sleep(0.5)

    #Tiempos_CH1.csv
    df_t1 = pd.DataFrame([freq_gen_fun, Unidades_tiempos_CH1])
    df_t1nuevo = pd.DataFrame(tiempos_CH1).T
    df_tiempos_CH1 = pd.concat([df_t1, df_t1nuevo], ignore_index=True)
    titulos_filas_4 = ["frecuencias generador funciones (Hz)", "Unidades tiempos CH1", "Tiempos CH1"]
    while len(titulos_filas_4) < len(df_tiempos_CH1):
        titulos_filas_4.append("")
    df_tiempos_CH1.insert(0, "Títulos de Fila", titulos_filas_4)
    df_tiempos_CH1.to_csv(r'C:.\Tiempos_CH1.csv', index=False, encoding = 'utf-8')
    time.sleep(0.5)

    #Tiempos_CH2.csv
    df_t2 = pd.DataFrame([freq_gen_fun, Unidades_tiempos_CH2])
    df_t2nuevo = pd.DataFrame(tiempos_CH2).T
    df_tiempos_CH2 = pd.concat([df_t2, df_t2nuevo], ignore_index=True)
    titulos_filas_5 = ["frecuencias generador funciones (Hz)", "Unidades tiempos CH2", "Tiempos CH2"]
    while len(titulos_filas_5) < len(df_tiempos_CH2):
        titulos_filas_5.append("")
    df_tiempos_CH2.insert(0, "Títulos de Fila", titulos_filas_5)
    df_tiempos_CH2.to_csv(r'C:.\Tiempos_CH2.csv', index=False, encoding = 'utf-8')
    time.sleep(0.5)

    print("Archivos guardados correctamente")

In [ ]:
#%% Itensidad en función de frecuencia

rm = pyvisa.ResourceManager()
rm.list_resources()

In [ ]:
resource_name_osciloscopio='USB0::0x0699::0x0363::C108013::INSTR'

osci = rm.open_resource(resource_name_osciloscopio)
print(osci.query('*IDN?'))

In [ ]:
#Creamos las distintas listas donde vamos a guardar las mediciones

señales_CH1 = []
señales_CH2 = []
tiempos_CH1 = []
tiempos_CH2 = []
freq_gen_fun = []
Unidades_señales_CH1 = []
Unidades_señales_CH2 = []
Unidades_tiempos_CH1 = []
Unidades_tiempos_CH2 = []
Escala_señales_CH1 = []
Escala_señales_CH2 = []
Escala_tiempos_CH1 = []
Escala_tiempos_CH2 = []
osci.write('DATa:WIDth 1')


In [ ]:

#mediciones = 10
#for freq in range(mediciones):

#Medimos canal 2
osci.write('DATa:SOUrce CH2')
#osci.write('AUTOSet EXECute')
time.sleep(1)

osci.write('DATa:ENCdg RPBinary')
xze2, xin2, yze2, ymu2, yoff2 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFf?;', separator=';')
Unidades_señales_CH2.append(osci.query('WFMPRE:YUNit?'))
Unidades_tiempos_CH2.append(osci.query('WFMPRE:XUNit?'))
Escala_señales_CH2.append(osci.query('CH2:SCAle?'))
Escala_tiempos_CH2.append(osci.query('HORizontal:MAIn:SCAle?'))

data2 = osci.query_binary_values('CURV?', datatype='B', container=np.array)
voltaje2 =(data2-yoff2)*ymu2+yze2;
tiempo2 = xze2 + np.arange(len(data2)) * xin2
señales_CH2.append(voltaje2)
tiempos_CH2.append(tiempo2)

#Medimos canal 1
osci.write('DATa:SOUrce CH1')
osci.write('DATa:ENCdg RPBinary')
xze1, xin1, yze1, ymu1, yoff1 = osci.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFf?;', separator=';')

Unidades_señales_CH1.append(osci.query('WFMPRE:YUNit?'))
Unidades_tiempos_CH1.append(osci.query('WFMPRE:XUNit?'))
Escala_señales_CH1.append(osci.query('CH1:SCAle?'))
Escala_tiempos_CH1.append(osci.query('HORizontal:MAIn:SCAle?'))

data1 = osci.query_binary_values('CURV?', datatype='B', container=np.array)
voltaje1 =(data1-yoff1)*ymu1+yze1;
tiempo1 = xze1 + np.arange(len(data1)) * xin1

señales_CH1.append(voltaje1)
tiempos_CH1.append(tiempo1)

time.sleep(1)
print("Señal adquirida")

print('FIN ( ˘▽˘)っ♨')


In [ ]:
#%%Adquisición

#(crear de antemano los 5 .csv con encoding utf-8):
#- Mediciones_CH1.csv
#- Mediciones_CH2.csv
#- Unidades y escala.csv
#- Tiempos_CH1.csv
#- Tiempos_CH2.csv
#poner direcciónes en cada código

guardar_mediciones()

# Análisis

(Función de análisis_espectro modificada para la señal davs)

devuelve: array de la señal normalizada y suave, eje x con las frecuencas relativas y eje x con las frecuencias absolutas,  
y los gráficos para ir ajustando los valores de `A,B,C,D` y los de `l` y `m` 

`espectro_suave_a`   
`f_rel_a`  
`f_abs_a`    

`n` cantidad de mediciones en el csv + 1  
`i` índice de la medicion a analizar (empezando desde 0)  
`a` sufijo para diferenciar mediciones  
`ruta` ruta de la carpeta con los 5 csv  
`A,B,C,D` límites para ajustar la lineal (para 'aplanar' la señal)  
`graficar` True muestra para ir variando los parámetros, False no los muestra  
`norm` True, normaliza la señal por su valor más alto, False no lo hace -> esto lo puse para, al variar el campo, ver si aumenta la absorción en cada pico.  
`l` índice de la lista de raíces (pasos por el cero de la parte pos a la neg) correspondiente al primer cero 'posta', o sea el que se corresponde con la primer línea de absorción  
`m` idem `l` pero para el último cero, la idea es usar -1, -2, etc, ya que no sabemos la cantidad de ceros de antemano

La función asume la señal del espectro en el canal 1


Cambios:
- no hacemos la señal toda positiva, justamente queremos ver los ceros
- no invertimos la señal en el eje y ni en el x:

estos parámetros ya no los uso:  
`h` altura a partir de la cual calcula los máximos (picos)  
`dis` distancia mínima en puntos entre picos  
`pro` "prominencia" mínima (cuán destacado es el pico)  



posible error al hacer el ajuste lineal:  
por la versión de `curvefit` a lo mejor el error en y tiene que ser un array y no sólo un valor, en ese caso añadir en sigma `error_Ch1_a * np.ones(len(tiempos_intervalos_a))`

In [3]:

def cargar_datos_2(n, i, ruta, a):
    
    #n número de datos tomados +1
    # i número de medición -1 (empezando a contar desde 1)
    # sufijo para cada medición distinta

    columnas = [f'{j}' for j in range(1, n)]

    # Rutas a archivos
    señales_Ch1 = os.path.join(ruta, 'Mediciones_CH1.csv')
    señales_Ch2 = os.path.join(ruta, 'Mediciones_CH2.csv')
    tiempos_Ch1 = os.path.join(ruta, 'Tiempos_CH1.csv')
    tiempos_Ch2 = os.path.join(ruta, 'Tiempos_CH2.csv')
    unidades_escalas = os.path.join(ruta, 'Unidades y escala.csv')

    # Lectura de archivos
    data_señales_Ch1 = pd.read_csv(señales_Ch1, skiprows=3, delimiter=',', header=None, names=columnas)
    data_señales_Ch2 = pd.read_csv(señales_Ch2, skiprows=3, delimiter=',', header=None, names=columnas)
    data_tiempos_Ch1 = pd.read_csv(tiempos_Ch1, skiprows=3, delimiter=',', header=None, names=columnas)
    data_tiempos_Ch2 = pd.read_csv(tiempos_Ch2, skiprows=3, delimiter=',', header=None, names=columnas)
    data_unidades_escalas = pd.read_csv(unidades_escalas, delimiter=',', header=None, names=columnas)

    # Selección de columna
    col_name = str(i + 1)

    # Variables con sufijo personalizado
    globals()[f'tiempos1_{a}'] = data_tiempos_Ch1[col_name].values
    globals()[f'voltajes1_{a}'] = data_señales_Ch1[col_name].values

    globals()[f'tiempos2_{a}'] = data_tiempos_Ch2[col_name].values
    globals()[f'voltajes2_{a}'] = data_señales_Ch2[col_name].values

    escala_Ch1 = float(data_unidades_escalas[col_name].iloc[4])
    escala_Ch2 = float(data_unidades_escalas[col_name].iloc[8])

    globals()[f'error_Ch1_{a}'] = escala_Ch1 * 10 / 256
    globals()[f'error_Ch2_{a}'] = escala_Ch2 * 10 / 256


def lineal(x, m, b):
  y =  m * x + b
  return y


def detectar_cruces_cero_negativos(tiempos, señal):
    cruces = []

    for i in range(len(señal) - 1):
        y1, y2 = señal[i], señal[i + 1]
        if y1 > 0 and y2 < 0:  # Cruce por cero con pendiente negativa
            t1, t2 = tiempos[i], tiempos[i + 1]
            # Interpolación lineal para encontrar el tiempo exacto del cruce
            t_cruce = t1 - y1 * (t2 - t1) / (y2 - y1)
            cruces.append(t_cruce)

    return np.array(cruces)


def graficar_6_subplots(x1, y1, x2, y2, x3, y3, x4, y4, x5, y5, x6, y6, zeros): #ahora en vez de peaks tengo zeros
    fig, axs = plt.subplots(3, 2, figsize=(10, 9))  # 3 filas, 2 columnas

    # Gráfico 1
    axs[0, 0].plot(x1, y1, '.', color='darkcyan', label='Canal 1')
    axs[0, 0].set_title("señal osciloscopio")
    axs[0, 0].set_xlabel("tiempo [s]")
    axs[0, 0].set_ylabel("v")

    # Gráfico 2
    axs[0, 1].plot(x2, y2, ".", color='darkslategray')
    axs[0, 1].set_title("recorte lineal")
    axs[0, 1].set_xlabel("tiempo [s]")
    axs[0, 1].set_ylabel("v")

    # Gráfico 3
    axs[1, 0].plot(x3, y3, ".", markersize=2, color='darkslategray')
    axs[1, 0].set_title("aplanado")
    axs[1, 0].set_xlabel("tiempo [s]")
    axs[1, 0].set_ylabel("davs")

    # Gráfico 4
    axs[1, 1].plot(x4, y4, ".", markersize=2, color='darkslategray')
    axs[1, 1].set_title("normalizado y suavizado")
    axs[1, 1].set_xlabel("tiempo [s]")
    axs[1, 1].set_ylabel("davs")

    # Gráfico 5
    axs[2, 0].plot(x5, y5, label="señal")
    axs[2, 0].plot(zeros, np.zeros_like(zeros), "rx",label="raíces detectadas")

    axs[2, 0].set_title("ceros")
    axs[2, 0].set_xlabel("tiempo [s]")
    axs[2, 0].set_ylabel("Davs")
    axs[2, 0].legend()  # <-- Mostrar leyenda sólo en el quinto gráfico

    # Gráfico 6
    axs[2, 1].plot(x6, y6, ".", markersize=2, color='teal')
    axs[2, 1].axvline(0 , color="darkslategray", linestyle="dashed") # para ver que haya tomado los ceros que queremos
    axs[2, 1].axvline(7.649, color="darkslategray", linestyle="dashed") # para ver que haya tomado los ceros que queremos
    axs[2, 1].set_title("reescalado")
    axs[2, 1].set_xlabel("Frecuencia relativa (Ghz)")
    axs[2, 1].set_ylabel("Davs")

    plt.tight_layout()
    plt.show()


def análisis_compacto_davs(n, i, a, ruta, A, B, C, D, graficar=True, norm=True,l = 0, m = -1):

        cargar_datos_2(n = n, i = i, ruta=ruta, a = 'a')

        #uso los dos extremos de la parte lineal para ajustar

        voltajes_intervalos_a = np.concatenate([voltajes1_a[A:B], voltajes1_a[C:D]])
        tiempos_intervalos_a = np.concatenate([tiempos1_a[A:B], tiempos1_a[C:D]])

        #Cuando se vea la recta, hago el ajuste:

        x_fit_a =  np.linspace(np.min(tiempos2_a), np.max(tiempos2_a), len(tiempos2_a), endpoint = True)

        param_iniciales = [0.01, 0]
        popt_0, pcov_0 = curve_fit(lineal, tiempos_intervalos_a,voltajes_intervalos_a, p0=param_iniciales, sigma = error_Ch1_a, absolute_sigma=True)
        incertidumbre_0 = np.sqrt(np.diag(pcov_0))

        tabla = pd.DataFrame({
                'Los parametros óptimos son': popt_0,
                'Los errores de los parámetros son': incertidumbre_0,
                })
        if graficar:#así oculto estas salidas
                print(tabla)


        t_espectro_a = tiempos1_a[B:C] #así ya está recortado
        espectro_a = (voltajes1_a - lineal(x_fit_a, *popt_0))[B:C] 

        #inertimos, para graficar la absorción en función del tiempo
        #Y normalizamos para que el pico de mayor amplitud valga 1
        #Además, hacemos que la señal sea toda positiva (así al comparar están todas en la misma escala vertical)
        
        #espectro_a_inv = espectro_a * (-1)
        #espectro_a_pos = espectro_a_inv - np.min(espectro_a_inv) # con el - me aseguro de si ya es toda positiva, que se baje el minimo al cero
        
        #normalizamos
        espectro_a_norm = espectro_a / np.max(espectro_a)

        #filtrado
        señal_suavizada_a = savgol_filter(espectro_a_norm, window_length=11, polyorder=2)
  
        #damos vuelta el espectro
        #espectro_suave_a = señal_suavizada_a[::-1]
        espectro_suave_a = señal_suavizada_a

        #encuentro el primer y último cero

        zeros = detectar_cruces_cero_negativos(t_espectro_a, espectro_suave_a)

        
        first_zero, last_zero = zeros[l], zeros[m] 
        #en general l = 0 y m = -1, peero si llega a detectar otros,
        #para poder ajustarlo y que no se reescalee mal


        # obtener primer y último cero
        x_first, x_last = first_zero, last_zero


        if graficar:#así oculto estas salidas
                print("Primer cero:", first_zero)
                print("Último cero:", last_zero)


        # reescaleo los tiempos a las frecuencias 
        # Fórmula de reescalado
        frecuencias_abs = [377.104391, 377.112040] #THz
        frecuencias_rel = [0, 7.649] #Ghz

        freq_relativas = (t_espectro_a - x_first) / (x_last - x_first) * (7.649 - 0) + 0 # le pongo el cero por si lo quiero cambiar después
        freq_absolutas = (t_espectro_a - x_first) / (x_last - x_first) * (frecuencias_abs[1] - frecuencias_abs[0]) + frecuencias_abs[0]


        if norm:
                espectro_suave_a = espectro_suave_a
        else:
                espectro_suave_a = espectro_suave_a * np.max(espectro_a)




        #grafico (sólo si graficar = True)
        if graficar:
                graficar_6_subplots(tiempos1_a, voltajes1_a, tiempos_intervalos_a, voltajes_intervalos_a, t_espectro_a, espectro_a, t_espectro_a, señal_suavizada_a, t_espectro_a, espectro_suave_a, freq_relativas, espectro_suave_a, zeros=zeros)


        #creo las variables globales

        globals()[f'espectro_suave_{a}'] = espectro_suave_a
        globals()[f'f_rel_{a}'] = freq_relativas
        globals()[f'f_abs_{a}'] = freq_absolutas

        #para ver el desplazamiento en función de la temperatura del láser
        #globals()[f't_raw_{a}'] = tiempos1_a
        #globals()[f'v_raw_{a}'] = voltajes1_a

        #para ver la amplitud relativa entre las señales, exporto la señal antes de normalizarla
        #señal_suavizada_a_sin_norm = savgol_filter(espectro_a_pos, window_length=11, polyorder=2) # la filtro
        #globals()[f'espectro_sin_norm_{a}'] = señal_suavizada_a_sin_norm


In [ ]:

análisis_compacto_davs(n=4, i=0, a = 'a', ruta='/Users/Mauri/Desktop/Labo 5/Espectroscopía láser/Med_comparación', A =500
                  , B = 1000, C = 1900, D = 2400, graficar=True,  norm=True, l = 0, m = -3)


# comparar a distintos c. magnéticos
También sirve para diferentes temperaturas

In [ ]:
análisis_compacto_davs(n=5, i=0, a = 'b', ruta='/Users/Mauri/Desktop/Labo 5/Espectroscopía láser/Med_davs', A =500
                  , B = 1000, C = 1900, D = 2400, graficar=False,  norm=True, l = 1, m = -1)

In [ ]:
análisis_compacto_davs(n=5, i=1, a = 'c', ruta='/Users/Mauri/Desktop/Labo 5/Espectroscopía láser/Med_davs', A =0
                  , B = 300, C = 1900, D = 2400, graficar=False,  norm=True, l = 0, m = -1)

In [ ]:
dominios = [f_rel_b, f_rel_c, f_rel_d, f_rel_e]
imagenes = [espectro_suave_b, espectro_suave_c, espectro_suave_d, espectro_suave_e]

campos_magnéticos = [25, 28, 30, 32]


#para ver las amplitudes relativas entre cada medición, 
#usando las imágenes obtenidas con norm = False!
#normalizo por el espectro más grande
valor_maximo = max(np.max(arr) for arr in imagenes) #es el mayor valor de todos los arrays
imagenes = [arr / valor_maximo for arr in imagenes] # esto divide a cada array de la lista por el valor_maximo


# Usamos un colormap para generar colores automáticamente
colormap = plt.cm.plasma  # plasma, magma, viridis, inferno, etc.
num_senales = len(dominios)
colors = [colormap(i  / (num_senales - 1) - 0.3) for i in range(num_senales)]

# Graficamos
plt.figure(figsize=(10, 6))

for i in range(num_senales - 1):
    plt.plot(dominios[i], imagenes[i], color=colors[i], label=f"T = {campos_magnéticos[i]} mT")

plt.title("Señal DAVS en función del c.m. aplicado")
plt.xlabel("Frecuencias relativas [Ghz]")
plt.ylabel("Señal DAVS [a.u.]")
plt.legend()
plt.grid(True, alpha = 0.6)
plt.show()
